In [1]:
#importing needed libraries

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import time as t


In [3]:
df_ratings = pd.read_csv('ratings_clean.csv')

In [ ]:
#checks how many ratings
num_users = len(df_ratings.userId.unique())
num_items = len(df_ratings.movieId.unique())
total_items = num_users * num_items

#new df to visualize rating counts
df_ratings_visual = pd.DataFrame(df_ratings.groupby('rating').size(), columns=['count'])


# removing the 0 column --> those would be missing ratings
df_ratings_cnt = df_ratings_visual[df_ratings_visual.index != 0.0].copy()
#used for highly skewed data
df_ratings_cnt['log_count'] = np.log1p(df_ratings_cnt['count'])

# plot
ax = df_ratings_cnt.reset_index().plot(
    x='rating',
    y='count',
    kind='bar',
    figsize=(12, 8),
    title='Distribution of rating scores',
    logy=True,
    fontsize=12,
    color='lightblue',
    edgecolor='black',
    alpha=0.7,
)

ax.set_xlabel("movie rating score")
ax.set_ylabel("number of ratings")


plt.show()



In [ ]:
# Read the data from the csv file
data = pd.read_csv('movies.csv')
data = data.drop('movieId', axis=1)
data = data.drop('title', axis=1)

#create dictionary for genres
genres = {
    'Action': 0,
    'Adventure': 0,
    'Animation': 0,
    'Children': 0,
    'Comedy': 0,
    'Crime': 0,
    'Documentary': 0,
    'Drama': 0,
    'Fantasy': 0,
    'Film-Noir': 0,
    'Horror': 0,
    'Musical': 0,
    'Mystery': 0,
    'Romance': 0,
    'Sci-Fi': 0,
    'Thriller': 0,
    'War': 0,
    'Western': 0,
    'IMAX': 0,
    '(no genres listed)': 0
}

# for each row in data, add the genres to the dictionary
start = t.time()
for row in data.iterrows():
    for genre in row[1]['genres'].split('|'):
        genres[genre] += 1
end= t.time()
print("time for scraping:",end - start)
            
plt.pie(genres.values(), labels=genres.keys(), autopct='%1.1f%%')
plt.axis('equal')
plt.show()

